In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import json
from keras.preprocessing.text import Tokenizer
import numpy as np 
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
from nltk.corpus import stopwords


In [3]:
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
with open("training_data.jsonl") as f:
    training_data = json.load(f)
with open("test_data.jsonl") as f:
    testing_data = json.load(f)
with open("validation_data.jsonl") as f:
    valid_data = json.load(f)

In [5]:
train_data = []

for i in range(len(training_data)):
    train_data.append(training_data[i][0][0] + ' ' + training_data[0][0][1] + ' ' + training_data[i][0][2])
    
print(len(train_data))

1386


In [6]:
val_data = []

for i in range(len(valid_data)):
    val_data.append(valid_data[i][0][0] + ' ' + valid_data[i][0][1] + ' ' + valid_data[i][0][2])
    
print(len(val_data))

175


In [7]:
REPLACE_BY_SPACE = re.compile('[/(){}\[\]\|@,;.-_-—]')

def clean_text(text):
    
    text = text.lower()
    text = REPLACE_BY_SPACE.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

train_data = list(map(clean_text, train_data))
val_data = list(map(clean_text, val_data))

print(len(train_data))
print(len(val_data))

1386
175


In [8]:
print(training_data[0][0][1])
print(train_data[0])

daphne ippolito | daniel duckworth | chris callison-burch | douglas eck
automatic detection generated text easiest humans fooled daphne ippolito daniel duckworth chris callison burch douglas eck recent advancements neural language modelling make possible rapidly generate vast amounts human sounding text capabilities humans automatic discriminators detect machine generated text large source research interest humans machines rely different cues make decisions perform careful benchmarking analysis three popular sampling based decoding strategies top k nucleus sampling untruncated random sampling show improvements decoding methods primarily optimized fooling humans comes expense introducing statistical abnormalities make detection easy automatic systems also show though human automatic detector performance improve longer excerpt length even multi sentence excerpts fool expert human raters % time findings reveal importance using human automatic detectors assess humanness text generation sys

In [9]:
diction = {}
for i in training_data:
    diction[i[1]] = 1
    
print(len(diction))

j = 1
for i in diction:
    diction[i] = j
    j += 1

print(diction)

20
{'generation': 1, 'sentiment analysis, stylistic analysis, and argument mining': 2, 'information extraction, retrieval and text mining': 3, 'resources and evaluation': 4, 'machine learning for nlp': 5, 'interpretability and analysis of models for nlp': 6, 'question answering': 7, 'linguistic theories, cognitive modeling and psycholinguistics': 8, 'discourse and pragmatics': 9, 'machine translation and multilinguality': 10, 'ethics in nlp': 11, 'summarization': 12, 'tagging, chunking, syntax and parsing': 13, 'dialogue and interactive systems': 14, 'semantics': 15, 'computational social science, social media and cultural analytics': 16, 'nlp applications': 17, 'language grounding to vision, robotics and beyond': 18, 'phonology, morphology and word segmentation': 19, 'speech and multimodality': 20}


In [10]:
train_labels = []
for i in range(len(training_data)):
    train_labels.append(diction[training_data[i][1]])
    
print(len(train_labels))

val_labels = []
for i in range(len(valid_data)):
    val_labels.append(diction[valid_data[i][1]])
    
print(len(val_labels))

1386
175


In [11]:
y_train = tf.convert_to_tensor(train_labels, dtype = tf.int64)
y_val = tf.convert_to_tensor(val_labels, dtype = tf.int64)

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
tokenizer.fit_on_texts(val_data)
print("Number of unique tokens", list(tokenizer.word_index.keys()))

Number of unique tokens ['model', 'models', 'daniel', 'chris', 'daphne', 'ippolito', 'duckworth', 'callison', 'burch', 'douglas', 'eck', 'language', 'data', 'task', 'based', 'neural', 'show', 'propose', 'learning', 'text', 'training', 'performance', 'tasks', 'information', 'state', 'results', 'paper', 'art', 'word', 'translation', 'two', 'dataset', 'approach', 'method', 'using', 'knowledge', 'new', 'domain', 'datasets', 'generation', 'methods', 'sentence', 'multi', 'attention', 'also', 'experiments', 'human', 'representations', 'work', 'graph', 'level', 'different', 'proposed', 'however', 'natural', 'machine', 'large', 'semantic', 'trained', 'existing', 'question', 'evaluation', 'context', 'embeddings', 'use', 'novel', 'framework', 'dialogue', 'problem', 'network', 'entity', 'first', 'words', 'cross', 'languages', 'better', 'sequence', 'previous', 'systems', 'end', 'e', 'english', 'multiple', 'well', 'pre', 'target', 'demonstrate', 'classification', 'document', 'present', 'one', 'appro

In [13]:
X_train = tokenizer.texts_to_sequences(train_data)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train)

X_val = tokenizer.texts_to_sequences(val_data)
X_val = tf.keras.preprocessing.sequence.pad_sequences(X_val, maxlen = X_train.shape[1])

print('Train data tensor shape', X_train.shape)
print('Validation data tensor shape', X_val.shape)

Train data tensor shape (1386, 194)
Validation data tensor shape (175, 194)


In [14]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(21, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 194, 100)          1085900   
                                                                 
 spatial_dropout1d (SpatialD  (None, 194, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 21)                2121      
                                                                 
Total params: 1,168,421
Trainable params: 1,168,421
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
epochs = 20
batch_size = 32
checkpoint_path = 'my_best_model.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (X_val,y_val),callbacks=[model_checkpoint_callback])

Epoch 1/20
44/44 [==============================] - 24s 423ms/step - loss: 2.9204 - accuracy: 0.0996 - val_loss: 2.8083 - val_accuracy: 0.1086
Epoch 2/20
44/44 [==============================] - 18s 414ms/step - loss: 2.8076 - accuracy: 0.1270 - val_loss: 2.7953 - val_accuracy: 0.1143
Epoch 3/20
44/44 [==============================] - 18s 418ms/step - loss: 2.6868 - accuracy: 0.2100 - val_loss: 2.7814 - val_accuracy: 0.1429
Epoch 4/20
44/44 [==============================] - 18s 411ms/step - loss: 2.3203 - accuracy: 0.3189 - val_loss: 2.4890 - val_accuracy: 0.2629
Epoch 5/20
44/44 [==============================] - 18s 410ms/step - loss: 1.9269 - accuracy: 0.4509 - val_loss: 2.4933 - val_accuracy: 0.2514
Epoch 6/20
44/44 [==============================] - 19s 426ms/step - loss: 1.5741 - accuracy: 0.5613 - val_loss: 2.4581 - val_accuracy: 0.2457
Epoch 7/20
44/44 [==============================] - 19s 434ms/step - loss: 1.1952 - accuracy: 0.6919 - val_loss: 2.4261 - val_accuracy: 0.2743

In [16]:
model.load_weights(checkpoint_path)

model.evaluate(X_val,y_val)

6/6 [==============================] - 0s 50ms/step - loss: 2.9722 - accuracy: 0.3657


[2.9721882343292236, 0.36571428179740906]

In [17]:
test_data = []

for i in range(len(testing_data)):
    test_data.append(testing_data[i][0][0] + ' ' + testing_data[0][0][1] + ' ' + testing_data[i][0][2])
    
print(len(test_data))

183


In [18]:
X_test = tokenizer.texts_to_sequences(test_data)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen = X_train.shape[1])
print('Test data tensor shape', X_test.shape)

Test data tensor shape (183, 194)


In [19]:
test_labels = []
for i in range(len(testing_data)):
    test_labels.append(diction[testing_data[i][1]])
    
print(len(test_labels))

y_test = tf.convert_to_tensor(test_labels, dtype = tf.int64)

183


In [20]:
model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 60ms/step - loss: 3.3035 - accuracy: 0.2787


[3.303474187850952, 0.2786885201931]

In [21]:
import gensim
from gensim.models import Word2Vec

w2vmodel = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP/GoogleNews-vectors-negative300.bin', binary=True)

print(w2vmodel)
# summarize vocabulary size in model
# words = list(embed.wv)
# print('Vocabulary size: %d' % len(words))


In [22]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

t_data = []
for idx in range(len(train_data)):
  x = nltk.tokenize.word_tokenize(train_data[idx])
  temp = []
  for token in x:
    # w2vmodel[i]
    if(token in w2vmodel):
      temp.append(w2vmodel[token])
  t_data.append(temp)


v_data = []
for idx in range(len(val_data)):
  x = nltk.tokenize.word_tokenize(val_data[idx])
  temp = []
  for token in x:
    # w2vmodel[i]
    if(token in w2vmodel):
      temp.append(w2vmodel[token])
  v_data.append(temp)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
t_data_pad = tf.keras.preprocessing.sequence.pad_sequences(t_data,dtype='float64')
print(len(t_data_pad[1]))
v_data_pad =tf.keras.preprocessing.sequence.pad_sequences(v_data, maxlen = t_data_pad.shape[1],dtype='float64')
print(len(v_data_pad[1]))

184
184


In [24]:
np.asarray(t_data_pad).shape

(1386, 184, 300)

In [25]:
model_w2v = Sequential()
# model_w2v.add(Embedding(len(tokenizer.word_index) + 1, 300, input_length = t_data_pad.shape[1]))
model_w2v.add(Input((184, 300 )))

model_w2v.add(SpatialDropout1D(0.2))
model_w2v.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model_w2v.add(Dense(21, activation = 'softmax'))
model_w2v.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model_w2v.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spatial_dropout1d_1 (Spatia  (None, 184, 300)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 21)                2121      
                                                                 
Total params: 162,521
Trainable params: 162,521
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
epochs = 100
batch_size = 32
checkpoint_path = 'best_w2v_model.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history_w2v = model_w2v.fit(t_data_pad, y_train, epochs = epochs, batch_size = batch_size, validation_data = (v_data_pad,y_val),callbacks=[model_checkpoint_callback])

Epoch 1/100
44/44 [==============================] - 25s 506ms/step - loss: 2.8695 - accuracy: 0.0960 - val_loss: 2.7712 - val_accuracy: 0.1714
Epoch 2/100
44/44 [==============================] - 21s 487ms/step - loss: 2.7370 - accuracy: 0.1674 - val_loss: 2.7365 - val_accuracy: 0.1371
Epoch 3/100
44/44 [==============================] - 21s 486ms/step - loss: 2.5965 - accuracy: 0.2208 - val_loss: 2.5494 - val_accuracy: 0.2286
Epoch 4/100
44/44 [==============================] - 22s 490ms/step - loss: 2.4955 - accuracy: 0.2417 - val_loss: 2.4747 - val_accuracy: 0.2514
Epoch 5/100
44/44 [==============================] - 22s 493ms/step - loss: 2.3814 - accuracy: 0.2785 - val_loss: 2.3597 - val_accuracy: 0.3200
Epoch 6/100
44/44 [==============================] - 22s 494ms/step - loss: 2.2629 - accuracy: 0.3276 - val_loss: 2.3807 - val_accuracy: 0.2800
Epoch 7/100
44/44 [==============================] - 25s 571ms/step - loss: 2.1886 - accuracy: 0.3348 - val_loss: 2.2964 - val_accuracy:

In [29]:
model_w2v.load_weights(checkpoint_path)

model_w2v.evaluate(v_data_pad,y_val)

6/6 [==============================] - 1s 130ms/step - loss: 2.6347 - accuracy: 0.4914


[2.6346640586853027, 0.49142858386039734]

In [30]:
ts_data = []
for idx in range(len(test_data)):
  x = nltk.tokenize.word_tokenize(test_data[idx])
  temp = []
  for token in x:
    # w2vmodel[i]
    if(token in w2vmodel):
      temp.append(w2vmodel[token])
  ts_data.append(temp)

ts_data_pad =tf.keras.preprocessing.sequence.pad_sequences(ts_data, maxlen = t_data_pad.shape[1],dtype='float64')
print(len(ts_data_pad[1]))

184


In [31]:
np.asarray(ts_data_pad).shape

(183, 184, 300)

In [32]:
model_w2v.evaluate(ts_data_pad,y_test)

6/6 [==============================] - 1s 137ms/step - loss: 2.3009 - accuracy: 0.5246


[2.3009350299835205, 0.5245901346206665]